#  10. Solidity 함수

* Last Updated 20200610WED1800 20190612WEB1200

## 10.1 학습내용

## 10.1.1 목표

* solidity 함수를 정의하고, 타입, 가시성을 설정하고 만들기
* modifier 만들기
* Event 발생하는 코드 구현
* fallback 설정하고 구현

###### 10.1.2 목차

* 10.1 생성자
* 10.2 함수 타잎
    * 10.2.1 view, pure 함수
    * 10.2.2 Payable 함수
    * 10.2.3 가시성
    * 실습 FunctionTest
* 10.3 modifier
    * 실습 UnderscoreTest
    * 실습 BankV3
* 10.4 Events and logs
    * 실습 testEvent
    * 실습 e_testDeposit
    * 실습 e_testDeposit
    * 문제 Multiply7Event.sol
    * 실습 e_testEvent 복수의 이벤트 사용하기
* 10.5 fallback
    * 실습 FallbackTest
    * 실습 Payable Fallback - Multiply7.sol

## 10. 함수

함수는 **제공되거나 built-in** 또는 **사용자가 정의 user-defined** 로 구분할 수 있다.
built-in 함수는 지금까지 배워왔던 ```web3.eth.getAccounts()``` 등 뿐만아리라 ```keccak256()```, ```sha256()```과 같은 여러 함수가 있다. 또한 우리가 스스로 만들어 사용할 수 있는데, 이를 사용자 정의함수라고 한다.

아래의 문법에 맞추어, 앞에 ```function```이라고 적어서 함수 선언을 한다.
함수의 입력과 출력을 선언한다.
가시성과 수행하는 기능의 성격에 따라 ```pure```, ```view```, ```payable```의 수식어를 붙여준다.

```python
function (param types) {internal|external} [pure|view|payable] [returns (return types)] varName;
```

## 10.1 생성자

생성자 ```constructor()```는 컨트랙의 객체를 생성한다.
컨트랙의 소유자를 설정하거나, 처음에 설정이 필요한 변수를 초기화할 경우 사용한다.
생성자는 객체를 생성할 때 (배포가 완료되는 시점) 한 번만 실행이 된다.
* 생성자는 구현하지 않아도 되며, 그럴 경우 기본 생성자가 주어진다. 기본 생성자에는 물론 arg가 없다.
* 생성자는 1개만 허용된다. overloading은 할 수 없다는 의미이다.
* 보통 외부에서 사용할 수 있도록 **public**으로 선언한다.
또는 **internal**로 선언하여 다른 컨트랙이 생성할 수 있게 한다.

```python
contructor() public { ... }
```


## 10.2 함수 타잎

### 10.2.1 view, pure 함수

보통 함수는 입출력에 따라 구분할 수 있다. 입력이 있는지, 반환 값이 무엇인지에 따라 구분할 수 있다.
블록체인 프로그램에서는 함수가 블록체인을 변경하는지에 따라 구분하고 있다.
상태를 변경하지 않는 함수는 ```view```, ```pure```로 표기해야 하고, 따라서 함수를 local에서 실행할 수 있게 한다.
**```view```**는 이전 ```constant```를 대체한 명령어로서, 상태를 수정하지 않는 경우, 즉 읽기 전용 read only 함수이다. 

**```pure```**는  ```view```보다 더 제약이 많은 함수에 붙이는 수식어이다. 함수의 지역변수만을 사용하며, 함수 밖의 그 어떤 변수를 수정하지 못할 뿐만 아니라, 상태 값을 읽지도 않는 경우에 사용한다. 

구분 | 설명
----------|----------
view | 상태를 수정하지 않는 경우
pure | 상태를 수정하지도 않으며, 읽지도 않는 경우

블록체인을 변경하는 **view, pure가 아닌 경우**에는 transaction이 발생하므로 **Transaction hash**가 반환된다. 즉 처리결과 값을 반환return 하지 않는다. event를 사용해서 값을 남겨진 로그에서 확인할 수 있다.

즉 ```pure``` 함수는 다음과 같은 경우는 할 수 없다.
* 상태변수 state variables를 읽거나
* 잔고 ```this.balance``` 또는 ```<address>.balance```를 읽는 경우
* 전역변수 ```block```, ```tx```, ```msg``` (```msg.sig```, ```msg.data``` 제외)를 읽는 경우
* ```pure```로 표기되지 않은 함수를 호출하는 경우
* ```opcodes```로 작성된 인라인 코드

또한 다음과 같은 경우는 블록체인을 수정하는 작업이기 때문에 ```view```는 사용할 수 없다.
- 상태변수를 저장하는 경우 (당연히 블록체인에 쓰는 경우이다)
- 이벤트 발생 (이벤트가 발생하면 로그에 기록이 남겨진다.)
- 다른 컨트랙 생성
- ```selfdestruct``` 컨트랙을 삭제하는 경우
- ```call``` 함수로 송금하는 경우
- ```view``` 또는 ```pure``` 아닌 함수를 호출하는 경우
- low-level calls을 호출하는 경우
- ```opcodes```로 작성된 인라인 코드


## 연습문제

앞서 작성했던 GlobalVarsTest.sol을 다시 돌아보자.
pure, view를 사용한 경우가 위에서 설명한 바와 같은지 확인하자.
송금하는 컨트랙도 잔고를 구할 경우 pure, view 어떤 수식어를 사용했는지 확인해보자. 


### 10.2.2 Payable 함수

```payable``` 함수는 Ether를 입출금하는 경우에 사용된다.
web3 호출하는 측에서 { value: 0 }에 ether를 입력하면 ```msg.value```로 전달된다.
컨트랙에서 호출하는 경우는 ```value()```를 사용한다.

### 10.2.3 가시성

* ```public``` 외부에서 누구나 사용할 수 있다.
* ```internal``` 기본default 가시성으로, 내부 또는 상속관계에서만 사용할 수 있다.
* ```private``` 자신만 사용할 수 있다.
* ```external```은 ```internal```의 반대로서 외부에서만 사용할 수 있다.

### 10.2.4 스타일

컨트랙 내에 함수를 배치할 때는 다음과 같은 순서로 적어준다 (참조 Solidity Style Guide)
- 생성자
- fallback 함수
- external 함수
- public 함수
- internal 함수
- private 함수

### 함수 실습 코드

줄 | 설명
-----|-----
4 ~ 7 ```constructor()``` | 생성자는 앞에 위치시킨다.
8 ~ 11 ```increatementX()``` | state var를 수정하므로 ```view```, ```pure```가 아님.
13 ~ 15 ```doubleX()``` | 내부함수 ```internal X2()```를 호출함. state var 수정하므로 ```view```, ```pure``` 아님.
17 ~ 19 ```divideBy(int by)``` | 0으로 나누어도 오류를 발생하지 않는다. 아직 소수는 지원하지 않으므로 필요한 경우 라이브러리를 사용한다.
21 ~ 23 ```getX_()``` | ```view```, ```pure```가 아니므로```returns(int)```해도 반환 값을 받지 못함. 해시값을 돌려줌.
25 ~ 27 ```getX()``` | state var를 읽으므로 ```view```를 사용한다.  ```pure```를 사용하지 않는다.
28 ~ 30 ```getBalance()``` | 잔고 조회는 view도 가능함.
32 ~ 33 ```deposit()``` | ```payable```로 선언해야 입금이 가능. 함수가 비워있어도 ```msg.value```를 입금.
35 ~ 37 ```X2()``` | ```internal```로 선언되어서 내부에서만 사용가능.
38 ~ 40 ```getBlockNumber()``` | 전역변수를 읽을 경우 ```view```

In [1]:
%%writefile src/FuctionTest.sol
pragma solidity ^0.6.0;
contract FunctionTest {
    int x;
    //constructor
    constructor() public {
        x = 0;
    }
    // updating state var x
    function incrementX() public {
        x += 1;
    }
    // call when x = 0
    function doubleX() public {
        X2();
    }
    // float not supported. try 0, 1/3...
    function divideBy(int by) view public returns(int) {
        return x/by;
    }
    // actually NONE returned
    function getX_() public returns(int) {
        return x;        
    }
    // try pure
    function getX() view public returns(int) {
        return x;
    }
    function getBalance() view public returns(uint) {
        return(address(this).balance);
    }
    // 'payable' means that msg.value can be deposited
    function deposit() public payable {
    }
    // can not be used in public
    function X2() internal {
        x *= 2;
    }
    function getBlockNumber() view public returns(uint) {
        return block.number;
    }
}

Writing src/FuctionTest.sol


## 10.3 modifier

함수의 **제약조건**으로 사용된다. 즉 어떤 함수가 실행되려면 만족해야 하는 조건을 modifier로 정할 수 있다. Owner인 경우, 일정한 기간에 포함되는 경우, 짝수에 해당하는 경우 등 조건을 걸어 그런 경우에만 함수를 실행할 수 있게 한다. 그 효과로 **if문을 제거하는 효과**가 있게 된다.

또한 modifier는 보통 함수가 그런 것처럼 **상속**이 가능하다. 또한 override할 수도 있다.


modifier를 사용할 때 underscore ```_``` 명령어를 통해 어디에 호출 소스코드가 포함되는지 분명히 할 수 있다.
아래 코드에서 포함되는 시점을 전 또는 후로 정할 수 있다.

In [2]:
%%writefile src/UnderscoreTest.sol
pragma solidity ^0.6.0;
contract UnderscoreTest {
    string season = "";
    function getSeason() view public returns(string memory) {
        return season;
    }
    function setWinter() public setSummerAfter {
        season = "winter";
    }
    function setSpring() public setSummerBefore {
        season = "spring";
    }
    modifier setSummerAfter() {
        season = "summer";
        _;
    }
    modifier setSummerBefore() {
        _;
        season = "summer";
    }
}

Writing src/UnderscoreTest.sol


## 실습: 은행

앞의 은행 코드에 fallabck, modifier를 넣어서 수정한다.

### 단계 1: 컨트랙 개발

* 입금을 연달아 할 경우, 반드시 일정시간이 지나야 (10초) 입금을 가능하게 하고
* 일정 잔고 이상일 경우에만 출금이 되도록 해보자.

In [17]:
%%writefile src/BankV3.sol
pragma solidity ^0.5.0;

contract BankV3 {
    address owner;
    uint balance;
    uint256 timeToDeposit;
    
    event PrintLog(string);
    event Sent(address from, address to, uint amount );
    constructor() public {
        owner = msg.sender;
        balance = 0;
    }
    function() external {
        emit PrintLog("Fallback called");
    }
    function forwardTo(address payable _receiver) public payable onlyOwner {
        //require(msg.sender == owner);
        _receiver.transfer(msg.value);
        emit Sent(msg.sender, _receiver, msg.value);
    }
    function getBalance() public view returns(uint, uint) {
        return (balance, address(this).balance);
    }
    function deposit(uint amount) public payable onlyAfter {
        timeToDeposit = now + 10 seconds;
        require(msg.value == amount);
        balance += amount;
    }
    function withdrawAll() public onlyOwner minBalance {
        balance -= address(this).balance;
        //require(msg.sender == owner);
        msg.sender.transfer(address(this).balance);
    }
    modifier onlyOwner {
        require(msg.sender == owner);
        _;
    }
    modifier onlyAfter {
        require(now >= timeToDeposit);
        _;
    }
    modifier minBalance {
        require(address(this).balance>101 wei);
        _;
    }
}

Overwriting src/BankV3.sol


### 단계 2: 컴파일

In [18]:
!solc --gas --abi --bin src/BankV3.sol


======= src/BankV3.sol:BankV3 =======
Gas estimation:
construction:
   20523 + 211800 = 232323
external:
   fallback:	1795
   deposit(uint256):	20509
   queryBalance():	601
   sendTo(address):	infinite
   withdraw():	infinite
Binary: 
608060405234801561001057600080fd5b50336000806101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff160217905550610423806100606000396000f3fe60806040526004361061005c576000357c01000000000000000000000000000000000000000000000000000000009004806336f40c61146100d35780633ccfd60b146100fe578063b6b55f2514610115578063e6d2524514610143575b34801561006857600080fd5b507f968f0302429ff0e5bd56a45ce3ba1f4fa79f4b822857e438616435f00c3b59fd60405180806020018281038252600f8152602001807f46616c6c6261636b2063616c6c6564000000000000000000000000000000000081525060200191505060405180910390a1005b3480156100df57600080fd5b506100e8610187565b6040518082815260200191505060405180910390f35b34801561010a57600080fd5b506101136101a6565b005b6101416004803603

### 단계 3: 컨트랙 배포

In [19]:
%%writefile src/BankV3Deploy__.js
var Web3=require('web3');
var web3;
if (typeof web3 !== 'undefined') {
    web3 = new Web3(web3.currentProvider);
} else {
    web3 = new Web3(new Web3.providers.HttpProvider("http://117.16.44.45:8445"));
}
var _abiArray=[{"constant":true,"inputs":[],"name":"queryBalance","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"withdraw","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[{"name":"_receiver","type":"address"}],"name":"sendTo","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"},{"payable":false,"stateMutability":"nonpayable","type":"fallback"},{"anonymous":false,"inputs":[{"indexed":false,"name":"","type":"string"}],"name":"PrintLog","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"name":"from","type":"address"},{"indexed":false,"name":"to","type":"address"},{"indexed":false,"name":"amount","type":"uint256"}],"name":"Sent","type":"event"}];
var _bin="608060405234801561001057600080fd5b50336000806101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff160217905550610423806100606000396000f3fe60806040526004361061005c576000357c01000000000000000000000000000000000000000000000000000000009004806336f40c61146100d35780633ccfd60b146100fe578063b6b55f2514610115578063e6d2524514610143575b34801561006857600080fd5b507f968f0302429ff0e5bd56a45ce3ba1f4fa79f4b822857e438616435f00c3b59fd60405180806020018281038252600f8152602001807f46616c6c6261636b2063616c6c6564000000000000000000000000000000000081525060200191505060405180910390a1005b3480156100df57600080fd5b506100e8610187565b6040518082815260200191505060405180910390f35b34801561010a57600080fd5b506101136101a6565b005b6101416004803603602081101561012b57600080fd5b8101908080359060200190929190505050610287565b005b6101856004803603602081101561015957600080fd5b81019080803573ffffffffffffffffffffffffffffffffffffffff1690602001909291905050506102b3565b005b60003073ffffffffffffffffffffffffffffffffffffffff1631905090565b6000809054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff163373ffffffffffffffffffffffffffffffffffffffff1614151561020157600080fd5b60653073ffffffffffffffffffffffffffffffffffffffff163111151561022757600080fd5b3373ffffffffffffffffffffffffffffffffffffffff166108fc3073ffffffffffffffffffffffffffffffffffffffff16319081150290604051600060405180830381858888f19350505050158015610284573d6000803e3d6000fd5b50565b600154421015151561029857600080fd5b600a420160018190555080341415156102b057600080fd5b50565b6000809054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff163373ffffffffffffffffffffffffffffffffffffffff1614151561030e57600080fd5b8073ffffffffffffffffffffffffffffffffffffffff166108fc349081150290604051600060405180830381858888f19350505050158015610354573d6000803e3d6000fd5b507f3990db2d31862302a685e8086b5755072a6e2b5b780af1ee81ece35ee3cd3345338234604051808473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1681526020018373ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168152602001828152602001935050505060405180910390a15056fea165627a7a723058201a36146d11a6f9cedca78eadc10bd6bb60f95c3202eb8ccd0253fb8389e02cbc0029";
var _contract = web3.eth.contract(_abiArray);
//unlock the account with a password provided
//web3.personal.unlockAccount(web3.eth.accounts[0],'password');
var _instance=_contract.new({data:"0x"+_bin,from:web3.eth.accounts[0],gas:1000000}, function(err, contract) {
    if (!err) {
        console.log("contractAddress: ", contract.address);
        console.log("transactionHash: ", contract.transactionHash);
    }
});

Overwriting src/BankV3Deploy.js


이번에는 ```async```, ```await```로 변경하여 코드를 작성해보자.
배포함수를 ```async function deploy()```로 선언한다.
그리고 그 안에 ```await web3.eth.getAccounts()```로 계정을 구해서 이를 활용한다.
컨트랙을 생성하기 위해 블록체인에 전송하는 ```await send()``` 함수도 비동기적으로 처리하여, 주소를 출력한다.

In [1]:
%%writefile src/BankV3Deploy.js
var Web3 = require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://117.16.44.45:8445"));
//var _abiArray = JSON.parse(_abiStr);
var _abiArray = [{"constant":true,"inputs":[],"name":"getBalance","outputs":[{"name":"","type":"uint256"},{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"_receiver","type":"address"}],"name":"forwardTo","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[],"name":"withdrawAll","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"},{"payable":false,"stateMutability":"nonpayable","type":"fallback"},{"anonymous":false,"inputs":[{"indexed":false,"name":"","type":"string"}],"name":"PrintLog","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"name":"from","type":"address"},{"indexed":false,"name":"to","type":"address"},{"indexed":false,"name":"amount","type":"uint256"}],"name":"Sent","type":"event"}];
var _bin = "0x" + "608060405234801561001057600080fd5b50336000806101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff160217905550600060018190555061046e806100686000396000f3fe608060405260043610610062576000357c0100000000000000000000000000000000000000000000000000000000900463ffffffff16806312065fe0146100d957806327d8ad881461010b578063853828b61461014f578063b6b55f2514610166575b34801561006e57600080fd5b507f968f0302429ff0e5bd56a45ce3ba1f4fa79f4b822857e438616435f00c3b59fd60405180806020018281038252600f8152602001807f46616c6c6261636b2063616c6c6564000000000000000000000000000000000081525060200191505060405180910390a1005b3480156100e557600080fd5b506100ee610194565b604051808381526020018281526020019250505060405180910390f35b61014d6004803603602081101561012157600080fd5b81019080803573ffffffffffffffffffffffffffffffffffffffff1690602001909291905050506101ba565b005b34801561015b57600080fd5b506101646102fe565b005b6101926004803603602081101561017c57600080fd5b8101908080359060200190929190505050610406565b005b6000806001543073ffffffffffffffffffffffffffffffffffffffff1631915091509091565b6000809054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff163373ffffffffffffffffffffffffffffffffffffffff1614151561021557600080fd5b8073ffffffffffffffffffffffffffffffffffffffff166108fc349081150290604051600060405180830381858888f1935050505015801561025b573d6000803e3d6000fd5b507f3990db2d31862302a685e8086b5755072a6e2b5b780af1ee81ece35ee3cd3345338234604051808473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1681526020018373ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168152602001828152602001935050505060405180910390a150565b6000809054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff163373ffffffffffffffffffffffffffffffffffffffff1614151561035957600080fd5b60653073ffffffffffffffffffffffffffffffffffffffff163111151561037f57600080fd5b3073ffffffffffffffffffffffffffffffffffffffff16316001600082825403925050819055503373ffffffffffffffffffffffffffffffffffffffff166108fc3073ffffffffffffffffffffffffffffffffffffffff16319081150290604051600060405180830381858888f19350505050158015610403573d6000803e3d6000fd5b50565b600254421015151561041757600080fd5b600a4201600281905550803414151561042f57600080fd5b806001600082825401925050819055505056fea165627a7a723058208ddb611672b7c3e783ed459f3e8c220ba0c80b8ee11280da9c22d97ca4916bfc0029";
//unlock the account with a password provided
//web3.personal.unlockAccount(web3.eth.accounts[0],'password');
async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin})
        .send({from: accounts[0], gas: 1000000}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        })
        //.then(function(newContractInstance){
        //    console.log(newContractInstance.options.address)
        //});
    console.log("---> The contract deployed to: " + deployed.options.address)
}
deploy()

Overwriting src/BankV3Deploy.js


In [3]:
!node src/BankV3Deploy.js

Deploying the contract from 0x21c704354D07f804baB01894e8B4eB4E0EBA7451
hash: 0x72da865e8f67e4fe5f0174beacc87f5757002abf3b500ba92d18bf77c061b756
(node:32444) UnhandledPromiseRejectionWarning: Error: Failed to check for transaction receipt:
{}
    at Object._fireError (/home/jsl/Code/git/bb/jsl/bitcoin/node_modules/web3-utils/src/index.js:57:17)
    at /home/jsl/Code/git/bb/jsl/bitcoin/node_modules/web3-core-method/src/index.js:275:27
    at <anonymous>
(node:32444) UnhandledPromiseRejectionWarning: Unhandled promise rejection. This error originated either by throwing inside of an async function without a catch block, or by rejecting a promise which was not handled with .catch(). (rejection id: 45)
(node:32444) [DEP0018] DeprecationWarning: Unhandled promise rejections are deprecated. In the future, promise rejections that are not handled will terminate the Node.js process with a non-zero exit code.


In [23]:
%%writefile src/BankV3Deploy.js
var Web3 = require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://117.16.44.45:8345"));
//var _abiArray = JSON.parse(_abiStr);
var _abiArray = [{"constant":true,"inputs":[],"name":"getBalance","outputs":[{"name":"","type":"uint256"},{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"_receiver","type":"address"}],"name":"forwardTo","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[],"name":"withdrawAll","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"},{"payable":false,"stateMutability":"nonpayable","type":"fallback"},{"anonymous":false,"inputs":[{"indexed":false,"name":"","type":"string"}],"name":"PrintLog","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"name":"from","type":"address"},{"indexed":false,"name":"to","type":"address"},{"indexed":false,"name":"amount","type":"uint256"}],"name":"Sent","type":"event"}];
var _bin = "0x" + "608060405234801561001057600080fd5b50336000806101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff160217905550600060018190555061046e806100686000396000f3fe608060405260043610610062576000357c0100000000000000000000000000000000000000000000000000000000900463ffffffff16806312065fe0146100d957806327d8ad881461010b578063853828b61461014f578063b6b55f2514610166575b34801561006e57600080fd5b507f968f0302429ff0e5bd56a45ce3ba1f4fa79f4b822857e438616435f00c3b59fd60405180806020018281038252600f8152602001807f46616c6c6261636b2063616c6c6564000000000000000000000000000000000081525060200191505060405180910390a1005b3480156100e557600080fd5b506100ee610194565b604051808381526020018281526020019250505060405180910390f35b61014d6004803603602081101561012157600080fd5b81019080803573ffffffffffffffffffffffffffffffffffffffff1690602001909291905050506101ba565b005b34801561015b57600080fd5b506101646102fe565b005b6101926004803603602081101561017c57600080fd5b8101908080359060200190929190505050610406565b005b6000806001543073ffffffffffffffffffffffffffffffffffffffff1631915091509091565b6000809054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff163373ffffffffffffffffffffffffffffffffffffffff1614151561021557600080fd5b8073ffffffffffffffffffffffffffffffffffffffff166108fc349081150290604051600060405180830381858888f1935050505015801561025b573d6000803e3d6000fd5b507f3990db2d31862302a685e8086b5755072a6e2b5b780af1ee81ece35ee3cd3345338234604051808473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1681526020018373ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168152602001828152602001935050505060405180910390a150565b6000809054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff163373ffffffffffffffffffffffffffffffffffffffff1614151561035957600080fd5b60653073ffffffffffffffffffffffffffffffffffffffff163111151561037f57600080fd5b3073ffffffffffffffffffffffffffffffffffffffff16316001600082825403925050819055503373ffffffffffffffffffffffffffffffffffffffff166108fc3073ffffffffffffffffffffffffffffffffffffffff16319081150290604051600060405180830381858888f19350505050158015610403573d6000803e3d6000fd5b50565b600254421015151561041757600080fd5b600a4201600281905550803414151561042f57600080fd5b806001600082825401925050819055505056fea165627a7a723058208ddb611672b7c3e783ed459f3e8c220ba0c80b8ee11280da9c22d97ca4916bfc0029";
//unlock the account with a password provided
//web3.personal.unlockAccount(web3.eth.accounts[0],'password');
async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin})
        .send({from: accounts[0], gas: 1000000}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        })
        //.then(function(newContractInstance){
        //    console.log(newContractInstance.options.address)
        //});
    console.log("---> The contract deployed to: " + deployed.options.address)
}
deploy()

Overwriting src/BankV3Deploy.js


In [24]:
!node src/BankV3Deploy.js

Deploying the contract from 0xa0EeCb43Bf5FB417fe6Fcd1Af14F757e89052ef1
hash: 0xb654da07e509ad39b40797b9892b2b0351ec6f962692b21f9fda2fa2e52d86e4
---> The contract deployed to: 0x290EbE3313d9538953c9e3dEB3aa52eAA7C0fb25


### 단계 4: 사용

프로그램에 설정한 제약조건을 지켜서 실행헤야 한다.
* 10초 이내 저축
* 잔고 101보다 적은데 출금

In [33]:
%%writefile src/BankV3Use__.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://117.16.44.45:8445"));
var _abiArray=[{"constant":true,"inputs":[],"name":"queryBalance","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"withdraw","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[{"name":"_receiver","type":"address"}],"name":"sendTo","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"},{"payable":false,"stateMutability":"nonpayable","type":"fallback"},{"anonymous":false,"inputs":[{"indexed":false,"name":"","type":"string"}],"name":"PrintLog","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"name":"from","type":"address"},{"indexed":false,"name":"to","type":"address"},{"indexed":false,"name":"amount","type":"uint256"}],"name":"Sent","type":"event"}];
var _contract = web3.eth.contract(_abiArray);
var bank = _contract.at("0x486d4a274e03639c2bd52369f38e15736d760ee0");
//var filter = bank.PrintLog(function (error, result) {
//  if (!error)
//    console.log(result);
//});
//console.log(bank.sendTo(0x778ea91cb0d0879c22ca20c5aea6fbf8cbeed480, 100,{from:web3.eth.accounts[0],gas:100000}));
console.log(bank.deposit(100, {from:web3.eth.accounts[0], value:100}));
console.log(bank.withdraw({from:web3.eth.accounts[0]}));
console.log(bank.queryBalance().toNumber());

Overwriting src/BankV3Use.js


블록체인에 send()가 필요한 함수는 비동기적으로 처리하기 위해 ```await```로 처리한다.
비동기적으로 처리하면, 예를 들어 입금 ```deposit()```하고 ```getBalance()```하면 잔고에 입금분만큼 반영이 되어있는 것을 알 수 있다.

In [44]:
%%writefile src/BankV3Use.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://117.16.44.45:8345"));
var _abiArray=[{"constant":true,"inputs":[],"name":"getBalance","outputs":[{"name":"","type":"uint256"},{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"_receiver","type":"address"}],"name":"forwardTo","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[],"name":"withdrawAll","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"},{"payable":false,"stateMutability":"nonpayable","type":"fallback"},{"anonymous":false,"inputs":[{"indexed":false,"name":"","type":"string"}],"name":"PrintLog","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"name":"from","type":"address"},{"indexed":false,"name":"to","type":"address"},{"indexed":false,"name":"amount","type":"uint256"}],"name":"Sent","type":"event"}];
var bank = new web3.eth.Contract(_abiArray, "0x290EbE3313d9538953c9e3dEB3aa52eAA7C0fb25");
//var filter = bank.PrintLog(function (error, result) {
//  if (!error)
//    console.log(result);
//});
//console.log(bank.sendTo(0x778ea91cb0d0879c22ca20c5aea6fbf8cbeed480, 100,{from:web3.eth.accounts[0],gas:100000}));

async function doIt() {
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
    bank.methods.getBalance().call().then(console.log);
    await bank.methods.deposit(111).send({from: accounts[0], value:111});
    bank.methods.getBalance().call().then(console.log);
    await bank.methods.withdrawAll().send({from: accounts[0]});    //greater than 101
    bank.methods.getBalance().call().then(console.log);
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
    
}
doIt()

Overwriting src/BankV3Use.js


In [45]:
!node src/BankV3Use.js

Account: 0xa0EeCb43Bf5FB417fe6Fcd1Af14F757e89052ef1
Balance before: 99973586239000000000
Result { '0': '0', '1': '0' }
Result { '0': '111', '1': '111' }
Result { '0': '0', '1': '0' }
Balance after: 99972227459000000000
Balance diff: 1358780000010240


결과는 비동기 함수를 일시에 실행하기 때문에 바르게 출력되지 않고 있다.
node창에서 실행을 해보면 알 수 있다.m

1. deposit(100);
2. withdraw() ---> 실패 (프로그램에 101 wei 이상 제약조건)
3. deposit(111);
value를 반드시 함수인자와 동일하게 입력해야 한다.
REMIX에서 할 경우에도 마찬가지이다. Run 탭 상단의 value, Deployed Contracts 함수의 인자 두 필드에 동일한 금액을 넣어준다.
4. 빠르게 (프로그램에 설정해 놓은 10초 이내) deposit(111) ---> 실패
5. queryBalance() --=> 211
6. deposit(111); ---> 10초가 지났으면 실행. 이때 마이닝을 해서 동기화를 시켜줄 필요가 있다.
7. queryBalance() ---> 322

```python
> bank.methods.getBalance().call().then(console.log);              잔고 0
> 0
> bank.methods.deposit(100).send({from:"0x4D2fF...", value:100});    입금 100
> //miner.start(1);admin.sleepBlocks(1);miner.stop();                마이닝
> bank.methods.getBalance().call().then(console.log);              입금금액 조회 100. 앞 입금거래를 마이닝하고 금액 증가.
> 100                                                                
> bank.methods.deposit(100).send({from:"0x4D2fF...", value:111});    입금 'value'와 '인자'가 서로 다르면 입금 실패
> //miner.start(1);admin.sleepBlocks(1);miner.stop();                마이닝
undefined
> bank.methods.getBalance().call().then(console.log);              실패하였으므로 잔고는 계속 100
> 100
> bank.methods.deposit(111).send({from:"0x4D2fF...", value:111});    입금 111
> //miner.start(1);admin.sleepBlocks(1);miner.stop();                마이닝
> bank.methods.getBalance().call().then(console.log);              입금 100+111 = 211
> 211
> bank.methods.deposit(111).send({from:"0x4D2fF...", value:111});    (10초 지나서) 입금 111, 마이닝하면 금액 증가함
> //miner.start(1);admin.sleepBlocks(1);miner.stop();
> bank.methods.getBalance().call().then(console.log);              입금 100+111+111 = 322
> 322
> bank.methods.deposit(111).send({from:"0x4D2fF...", value:111});    입금 111, 마이닝하면 금액 증가함
> bank.methods.deposit(111).send({from:"0x4D2fF...", value:111});    입금 111, 10초 이내이므로 마이닝해도 금액 증가하지 않음
> //miner.start(1);admin.sleepBlocks(1);miner.stop();
> bank.methods.getBalance().call().then(console.log);              여러번 입금해도 10초 이내 거래는 실패하고 잔고가 늘지 않음
> 433
```

## 10.4 Events and logs
<a id='S.3.5'></a>

컴퓨터에서 이벤트는 마우스, 키보드 또는 센서로부터 발생할 수 있다.
블록체인에서는 그 특성 상 마우스나 키보드의 이벤트가 발생할 수 없고,
어떤 함수가 실행되는 경우 이벤트가 발생되는 것으로 한다.

###  event, emit

함수가 호출되고 함수 내의 **```emit```**명령어로 **이벤트가 발생**된다.
그러면 이벤트는 **로그**에 기록되고, 그로부터 이벤트가 인식된다.
EVM의 로그 기능을 사용한다. 로그는 blockchain에 저장된다.
이벤트는 **상속**할 수 있어서, 부모가 정의한 이벤트를 자식이 물려받아 발생시킬 수 있다.

#### 단계 1. 이벤트 설정
CapWords 스타일로 단어 첫 글자를 대문자로 적어준다.
인자를 넣어줄 수 있다.
인자 3개까지 로그에서 인덱스로 사용할 수 있고,
인자의 해시 값이 로그의 ```topic```으로 출력된다.
```python
event PrintLog()
```

#### 단계 2. 함수를 호출할 때 이벤트가 발생하도록 연결한다. 이런 연결을 바인딩 **binding**이라고 한다.

```python
function fireEvent() {
    emit PrintLog()
    ...
}
```

#### 단계 3. 이벤트의 발생을 callback 함수로 리스닝한다. ```instance.events.PrintLog()```, 인자가 있어도 생략할 수 있다.
JavaScript callbacks을 사용하여 이벤트가 호출되는지 리스닝한다.

```python
1) 이벤트를 생성하고 리스닝한다.
> var myEvent = myInstance.events.PrintLog({from: web3.eth.accounts[0]}, {
    fromBlock: 0,     //시작 블록 수
    toBlock: 'latest' //끝 블록 수
  });
> myEvent.watch(function (error, result) {
    if (!error) {
        console.log("Event triggered ===> ",result);
        process.exit(1);
    }
});
또는 한 명령어로 합쳐서 다음과 같이 리스닝할 수 있다.
var myEvent = myInstance.events.PrintLog({fromBlock: 0}, fromfunction(error, event) {
    if (!error)
        console.log(event);
});
```


#### 단계 4. 함수호출로 이벤트 발생.
이벤트를 가지고 있는 함수 호출, ```_instance.MyFunction()``` or ```_instance.MyEvent.watch()```

```python
> myInstance.fireEvent();
```

#### 단계 5. 이벤트 watch 중지
```python
> myEvent.stopWatching();
```

## 예제: 간단한 이벤트

이벤트를 만들고 함수가 호출되는 시점에 발생하도록 해보자.
이벤트는 발생하는 시점에 "Hello World!" 문자열이 출력되도록 한다.
이벤트가 발생하는지는 클라이언트에서 리스닝해서 알아 낸다.
이벤트가 발생하면 로그에 기록이 되고, 리스닝한다는 것은 이런 로그의 기록에 이벤트가 발생했는지를 확인하는 것이다.

### 단계 1: 컨트랙 개발

In [1]:
%%writefile src/EventTest.sol
pragma solidity ^0.6.0;
contract EventTest {
    event MyLog(string my);
    function myFunction() public {
        emit MyLog("Hello World!");
    }
}

Overwriting src/EventTest.sol


### 단계 2: 컴파일

In [1]:
!solc --abi --bin --gas src/EventTest.sol


======= src/EventTest.sol:EventTest =======
Gas estimation:
construction:
   93 + 42600 = 42693
external:
   myFunction():	1748
Binary:
6080604052348015600f57600080fd5b5060d58061001e6000396000f3fe6080604052348015600f57600080fd5b506004361060285760003560e01c8063c3780a3a14602d575b600080fd5b60336035565b005b7f5186edd9beca61d795526ca1f274260b3fc74be3e10e1f02e1be1552e14f137360405180806020018281038252600c8152602001807f48656c6c6f20576f726c6421000000000000000000000000000000000000000081525060200191505060405180910390a156fea2646970667358221220b88a03b3a7cef85cfe2ce5e017bedbe9e192f6ef40c8dbc84a9082af5a23d4c164736f6c63430006010033
Contract JSON ABI
[{"anonymous":false,"inputs":[{"indexed":false,"internalType":"string","name":"my","type":"string"}],"name":"MyLog","type":"event"},{"inputs":[],"name":"myFunction","outputs":[],"stateMutability":"nonpayable","type":"function"}]


### 단계 3: 컨트랙 배포

In [29]:
%%writefile src/EventTestDeploys__.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://117.16.44.45:8445"));
//copy abi from '!solc --abi src/testEvent.sol'
var _abiArray=[{"constant":false,"inputs":[],"name":"MyFunction","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"anonymous":false,"inputs":[{"indexed":false,"name":"","type":"string"}],"name":"MyLog","type":"event"}];
//var _abiArray=JSON.parse(_abiStr);
//copy bin from '!solc --bin src/testEvent.sol'
var _bin="0x"+"6080604052348015600f57600080fd5b5060e88061001e6000396000f3fe6080604052348015600f57600080fd5b50600436106045576000357c010000000000000000000000000000000000000000000000000000000090048063ad6e81d214604a575b600080fd5b60506052565b005b7f5186edd9beca61d795526ca1f274260b3fc74be3e10e1f02e1be1552e14f137360405180806020018281038252600c8152602001807f48656c6c6f20576f726c6421000000000000000000000000000000000000000081525060200191505060405180910390a156fea165627a7a72305820da9dac7ab71e948c4150d99274d7b603a5877bd7f86eb48ce48f0f58aad0ba280029";
var _contract = web3.eth.contract(_abiArray);
var _instance=_contract.new({data:_bin,from:web3.eth.accounts[0],gas:1000000}, function(err, contract) {
  if (!err && contract.address)
    console.log("contractAddress: ", contract.address);
    console.log("transactionHash: ", contract.transactionHash);
});
console.log("after async - transactionHash: ",_instance.transactionHash)
console.log("after async - contractAddress: ",_instance.address);

Overwriting src/testEvent1.js


In [3]:
%%writefile src/EventTestDeploy.js
var Web3 = require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://117.16.44.45:8345"));
//var _abiArray = JSON.parse(_abiStr);
var _abiArray  =[{"anonymous":false,"inputs":[{"indexed":false,"internalType":"string","name":"my","type":"string"}],"name":"MyLog","type":"event"},{"inputs":[],"name":"myFunction","outputs":[],"stateMutability":"nonpayable","type":"function"}];
var _bin = "0x" + "6080604052348015600f57600080fd5b5060d58061001e6000396000f3fe6080604052348015600f57600080fd5b506004361060285760003560e01c8063c3780a3a14602d575b600080fd5b60336035565b005b7f5186edd9beca61d795526ca1f274260b3fc74be3e10e1f02e1be1552e14f137360405180806020018281038252600c8152602001807f48656c6c6f20576f726c6421000000000000000000000000000000000000000081525060200191505060405180910390a156fea2646970667358221220b88a03b3a7cef85cfe2ce5e017bedbe9e192f6ef40c8dbc84a9082af5a23d4c164736f6c63430006010033";

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin})
        .send({from: accounts[0], gas: 364124, gasPrice: '1000000000'}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        })
        //.then(function(newContractInstance) {
        //    console.log(newContractInstance.options.address)
        //});
    console.log("---> The contract deployed to: " + deployed.options.address)
}
deploy()

Overwriting src/EventTestDeploy.js


사설망에 배포할 경우에는 계정 암호 풀어야 ```authentication needed: password or unlock``` 오류가 발생하지 않는다.

In [4]:
!node src/EventTestDeploy.js

Deploying the contract from 0x28E9D5b10083b74406b50cbb7B48a299BA6C8Be1
hash: 0xc301b9cb2c7d6f0f98bc56370a34f46174867969c5b5f2cef9baa7e5619dd153
---> The contract deployed to: 0x84E260576E134A90a54e35d526a9FCB83Fd4D6EC


위에서 주어진 ```transactionHash```를 가지고 처리결과를 알 수 있다.
gas 사용량, contractAddress도 찾을 수 있다.

In [6]:
!geth --exec 'eth.getTransactionReceipt("0xf146911d309a2039244e57335f121bc52ea917bd23547e1801c331d941551e0c");' attach http://117.16.44.45:8345

{
  blockHash: "0x3ac5bd8bcb62a46a8338a6585d01afd6a6e5201360491d633ee322ffd9432491",
  blockNumber: 1,
  contractAddress: "0x577f7d6f4f72cd57cfd3bbe7eec685ce367d4903",
  cumulativeGasUsed: 110297,
  from: "0x29ce1c8ad6a62b0e065890dd1eb0ff86da435929",
  gasUsed: 110297,
  logs: [],
  logsBloom: "0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000",
  r: "0xbf9f397bbdb8ec41e62bd274668ab297d3ab6a90ea124571c28fb74edfa805bc",
  s: "0x23a01b896e7df3386d9a3581afa02117122d3f3474482cb5d1986d34bbadcb94",
  status: "0x1",
  to: null,
  transa

### 단계 4: 사용


In [1]:
%%writefile src/testEvent2__.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://117.16.44.45:8445"));
var _abiArray=[{"constant":false,"inputs":[],"name":"MyFunction","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"anonymous":false,"inputs":[{"indexed":false,"name":"","type":"string"}],"name":"MyLog","type":"event"}];
var _contract=web3.eth.contract(_abiArray);
var _address="0x40dc58fc998d6ed4a384e01d89ba297d5bea2863";
var _instance=web3.eth.contract(_abiArray).at(_address);
var filter = _instance.MyLog(function (error, result) {
  if (!error)
    console.log(result);
});
_instance.MyFunction.sendTransaction({from:web3.eth.accounts[0]});

Overwriting src/testEvent2.js


```JSON.stringify```는 Object의 내용을 문자열로 출력한다.

In [1]:
%%writefile src/EventTestHttpNoEventFiredUse.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://117.16.44.45:8345"));
var _abiArray = [{"anonymous":false,"inputs":[{"indexed":false,"internalType":"string","name":"my","type":"string"}],"name":"MyLog","type":"event"},{"inputs":[],"name":"myFunction","outputs":[],"stateMutability":"nonpayable","type":"function"}];
var _test = new web3.eth.Contract(_abiArray, '0x84E260576E134A90a54e35d526a9FCB83Fd4D6EC');
var event = _test.events.MyLog({fromBlock: 0}, function (error, result) {
    if (!error) {
        console.log("Event fired: " + JSON.stringify(result.returnValues));
    }
});

async function doIt() {
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
    const value = await _test.methods.myFunction()
        .send({from: accounts[0], gas: 364124, gasPrice: '1000000000'})
        //.then(function(value) {console.log("---> myFunction called " + JSON.stringify(value.events.MyLog.returnValues));});
    console.log("---> myFunction called " + JSON.stringify(value.events.MyLog.returnValues));
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
}
doIt()

Overwriting src/EventTestHttpNoEventFiredUse.js


HttpProvider는 Event를 호출하지 못하고 있다. 함수 호출하면서 발생한 로그를 ```value.events.MyLog.returnValues```를 통해 출력할 수 있다.

In [2]:
!node src/EventTestHttpNoEventFiredUse.js

Account: 0x28E9D5b10083b74406b50cbb7B48a299BA6C8Be1
Balance before: 99932263510999996667
---> myFunction called {"0":"Hello World!","my":"Hello World!"}
Balance after: 99932240490999996667
Balance diff: 23019999985664


#### WebSocketProvider

위 프로그램에서 HttpProvider --> WebsocketProvider으로 변경한다. 사설망을 사용하는 경우, WebSocket이 가능하도록 설정해야 한다.

#### 파일에 로그 쓰기

이벤트가 발생하면 로컬에 다를 프로세스를 호출하는 작업을 할 수 있다. 
```fs.writeFile(파일명, 데이터, 인코딩방식, callback함수)```함수를 사용하여 이벤트 발생 로그를 로컬 파일에 써보자.
```fs.appendFile()```은 파일에 추가하는 기능이다.

In [1]:
%%writefile src/EventTestWsUse.js
var Web3=require('web3');
var fs=require('fs');
var web3 = new Web3(new Web3.providers.WebsocketProvider("http://117.16.44.45:8345"));
var _abiArray=[{"anonymous":false,"inputs":[{"indexed":false,"internalType":"string","name":"my","type":"string"}],"name":"MyLog","type":"event"},{"inputs":[],"name":"myFunction","outputs":[],"stateMutability":"nonpayable","type":"function"}];

async function doIt() {
    var _test = new web3.eth.Contract(_abiArray, '0x84E260576E134A90a54e35d526a9FCB83Fd4D6EC');
    var event = _test.events.MyLog({fromBlock: 0}, function (error, result) {
        if (!error) {
            log = JSON.stringify(result.returnValues);
            console.log("Event fired: " + log);
            //fs.writeFile("src/EventTestLog.txt", log, "utf-8", function(e) {
            fs.appendFile("src/EventTestLog.txt", log, "utf-8", function(e) {
                if(!e) {
                    console.log(">> Writing to file");
                }
            });
        }
    });
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
    const value = await _test.methods.myFunction()
        .send({from: accounts[0], gas: 364124, gasPrice: '1000000000'})
        //.then(function(value) {console.log("---> myFunction called " + JSON.stringify(value.events.MyLog.returnValues));});
    console.log("---> myFunction called " + JSON.stringify(value.events.MyLog.returnValues));
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
}

doIt()

Overwriting src/EventTestWsUse.js


계정 암호 해제해야 한다. (node:19884) UnhandledPromiseRejectionWarning: Error: Returned error: authentication needed: password or unlock


In [ ]:
!node src/EventTestWsUse.js

Event fired: {"0":"Hello World!","my":"Hello World!"}
Account: 0x28E9D5b10083b74406b50cbb7B48a299BA6C8Be1
>> Writing to file
Balance before: 99885635031999993577
Event fired: {"0":"Hello World!","my":"Hello World!"}
>> Writing to file
---> myFunction called {"0":"Hello World!","my":"Hello World!"}
Balance after: 99885612011999993577
Balance diff: 23020000002048


이벤트 발생하면서 파일에는 아래와 같이 JSON 형식으로 작성된다.

In [1]:
!cat src/EventTestLog.txt

{"0":"Hello World!","my":"Hello World!"}

### event indexing

매개변수를 인덱싱한다는 것은, 빠른 검색을 위해 색인을 만들어 놓는다는 의미이다.
매개변수 3개까지는 인덱싱을 할 수 있다.

예를 들어, Transfer(address indexed _from, address indexed _to, uint256 _value) 이렇게 인덱스가 잡혀 있다면
이 경우 전체가 아니라, 송신주소, 수신주소로 국한지어 검색할 수 있게 된다.
```python
contract.Event({from:"0xc32c36e7cea09c00e0dbd4bb4e72b8baaa51fd45"},{fromBlock: 0, toBlock: 'latest'});
```

### event overloading

컨트랙에 이벤트를 여러 개 가질 수 있는지의 문제이다.
이벤트 **오버로딩**도 가능한지 해보자.

## 실습: Order

### 단계 1: 컨트랙 개발

변수 ```_itemId```는 ```bytes2```으로 선언되어있다. 값을 입력할 경우 16진수라서 4자리를 넣었다 (예: "0x1234")


In [2]:
%%writefile src/OrderEvent.sol
pragma solidity ^0.6.0;

contract OrderEvent {
    uint unitPrice = 10;
    event OrderLog(string);
    event OrderLog(bytes2 _itemId, uint _value);
    event OrderLog(uint256 timestamp);
    event OrderLog(address indexed _from, bytes2 _itemId, uint indexed _value);

    function order(bytes2 _itemId, uint quantity) public payable {
        uint256 orderTime = now;
        uint256 orderAmount = quantity * unitPrice;
        require(msg.value == orderAmount);
        emit OrderLog("Ordered");
        emit OrderLog(orderTime);
        emit OrderLog(msg.sender, _itemId, msg.value);
    }
}

Overwriting src/OrderEvent.sol


### 단계 2: 컴파일

In [3]:
!solc --abi --bin --gas src/OrderEvent.sol


======= src/OrderEvent.sol:OrderEvent =======
Gas estimation:
construction:
   20159 + 102600 = 122759
external:
   order(bytes2,uint256):	5601
Binary:
6080604052600a60005534801561001557600080fd5b50610201806100256000396000f3fe60806040526004361061001e5760003560e01c806332af7f7314610023575b600080fd5b61007a6004803603604081101561003957600080fd5b8101908080357dffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff191690602001909291908035906020019092919050505061007c565b005b6000429050600080548302905080341461009557600080fd5b7fb9e815be6bf0b588396af8fa0244c9f92e26ad4dbf3694c09d90b0cc9f2be8e56040518080602001828103825260078152602001807f4f7264657265640000000000000000000000000000000000000000000000000081525060200191505060405180910390a17f2fb1e4e17675a393b0a43253759334f7e61806d1ac2690f0432362ed1c4c1611826040518082815260200191505060405180910390a1343373ffffffffffffffffffffffffffffffffffffffff167f1f7c3420c50f5b8875456d867e023a6d440ef2c1706d52ba38f5c6f553b53d508660405180827dfffffffffffffffffffffffffff

### 단계 3: 컨트랙 배포

컴파일하고 출력되는 abi, bin을 복사해서 배포 프로그램에 넣어주자.
사설망에 배포하려면, 계정을 unlock하는 것을 잊지 말자.

```python
> personal.unlockAccount(eth.accounts[0]);
Unlock account 0x21c704354d07f804bab01894e8b4eb4e0eba7451
Passphrase: 
```


In [12]:
%%writefile src/OrderEventDeploy.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://117.16.44.45:8345"));
var _abiArray = [{"anonymous":false,"inputs":[{"indexed":false,"internalType":"string","name":"","type":"string"}],"name":"OrderLog","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"bytes2","name":"_itemId","type":"bytes2"},{"indexed":false,"internalType":"uint256","name":"_value","type":"uint256"}],"name":"OrderLog","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"uint256","name":"timestamp","type":"uint256"}],"name":"OrderLog","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"_from","type":"address"},{"indexed":false,"internalType":"bytes2","name":"_itemId","type":"bytes2"},{"indexed":true,"internalType":"uint256","name":"_value","type":"uint256"}],"name":"OrderLog","type":"event"},{"inputs":[{"internalType":"bytes2","name":"_itemId","type":"bytes2"},{"internalType":"uint256","name":"quantity","type":"uint256"}],"name":"order","outputs":[],"stateMutability":"payable","type":"function"}];
//var _abiArray = JSON.parse(_abiStr);
var _bin="0x" + "6080604052600a60005534801561001557600080fd5b50610201806100256000396000f3fe60806040526004361061001e5760003560e01c806332af7f7314610023575b600080fd5b61007a6004803603604081101561003957600080fd5b8101908080357dffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff191690602001909291908035906020019092919050505061007c565b005b6000429050600080548302905080341461009557600080fd5b7fb9e815be6bf0b588396af8fa0244c9f92e26ad4dbf3694c09d90b0cc9f2be8e56040518080602001828103825260078152602001807f4f7264657265640000000000000000000000000000000000000000000000000081525060200191505060405180910390a17f2fb1e4e17675a393b0a43253759334f7e61806d1ac2690f0432362ed1c4c1611826040518082815260200191505060405180910390a1343373ffffffffffffffffffffffffffffffffffffffff167f1f7c3420c50f5b8875456d867e023a6d440ef2c1706d52ba38f5c6f553b53d508660405180827dffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff19167dffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff1916815260200191505060405180910390a35050505056fea2646970667358221220f1639089861eff944a29e48ea7925f02376e324f0d6656fb1cff157211e5e3d564736f6c63430006010033";

//unlock the account with a password provided
//web3.personal.unlockAccount(web3.eth.accounts[0],'password');
async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin})
        .send({from: accounts[0], gas: 300000}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        })
        //.then(function(newContractInstance) {
        //    console.log(newContractInstance.options.address)
        //});
    console.log("---> The contract deployed to: " + deployed.options.address)
}

deploy()

Overwriting src/OrderEventDeploy.js


In [13]:
!node src/OrderEventDeploy.js

Deploying the contract from 0x28E9D5b10083b74406b50cbb7B48a299BA6C8Be1
hash: 0x8ad8ecc07dde85a2a5205c94a24bde80d0335d5e36c6cf24dd1c3b70899b112c
---> The contract deployed to: 0x759575cbf8D032b5715a61412fbf7c0e8b729940


In [1]:
!geth --exec "eth.getTransactionReceipt('0x77ec35fe3c55b792b6d9b998fa6e6803a3da28fcf17ef07ca5bdbe996f69d9b5')" attach http://117.16.44.45:8345

{
  blockHash: "0xf886f9114d3735207d53b43306ea2aa06e0efa53d744a8e64f2edfd6106eac87",
  blockNumber: 1,
  contractAddress: "0xa4eeb56806ad9fca1f53fe44d38f27cb7cac1fed",
  cumulativeGasUsed: 201359,
  from: "0xe1cd15d96c6109d32b542573f2943878cc0f5b95",
  gasUsed: 201359,
  logs: [],
  logsBloom: "0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000",
  r: "0x2403903754ecbc39258859b7aaa8e5b48141193fbaf410af3da8da5bdd1febaf",
  s: "0x5d147727cb64c69d24307e1a47e157b06f980418d3c4f2cf1eae4aa796ced744",
  status: "0x1",
  to: null,
  transa

### 단계 4: 사용

event는 async방식으로 ```order()```함수가 호출되면 실행된다.
Order에 넘겨주는 매개변수는 바이트는 2바이트를 채워서 "0x1234", 주문량은 정수 3으로 맞추어 준다.
호출한 클라이언트에 event의 argument를 받아볼 수 있다 (예: result.args._from은 주소)

currentProvider는 MetaMask, Mist와 같은 Wallet을 사용하는 경우 자동으로 설정된다.

라인 | 설명
-----|-----
6 | WebsocketProvider으로 설정한다.
17 | **```filter: {_from: accounts[0], _value: 30}```** 인덱싱 걸어놓은 이벤트에 대해 해당 조건에 맞는 경우만 출력한다. 따라서 40, 100은 출력하지 않는다.
18 | **```fromBlock: 'latest', toBlock: 'pending'```** 최근부터 대기하는 이벤트를 출력한다. fromBlock: 0, toBlock: 'latest'는 처움 블록부터 발생한 이벤트를 모두 출력한다.
35 ~ 39 | 이벤트 로그에 대해 인덱싱에 해당되지 않아, ```my.events.OrderLog.returnValues``` 필드가 없게 되어 ```undefined```로 출력하고 있다.

In [1]:
%%writefile src/OrderEventUse.js
var Web3=require('web3');
var web3;
if (typeof web3 !== 'undefined') {
    web3 = new Web3(web3.currentProvider);
} else {
    var web3 = new Web3(new Web3.providers.WebsocketProvider("http://117.16.44.45:8345"));
}
var _abiArray = [{"anonymous":false,"inputs":[{"indexed":false,"internalType":"string","name":"","type":"string"}],"name":"OrderLog","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"bytes2","name":"_itemId","type":"bytes2"},{"indexed":false,"internalType":"uint256","name":"_value","type":"uint256"}],"name":"OrderLog","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"uint256","name":"timestamp","type":"uint256"}],"name":"OrderLog","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"_from","type":"address"},{"indexed":false,"internalType":"bytes2","name":"_itemId","type":"bytes2"},{"indexed":true,"internalType":"uint256","name":"_value","type":"uint256"}],"name":"OrderLog","type":"event"},{"inputs":[{"internalType":"bytes2","name":"_itemId","type":"bytes2"},{"internalType":"uint256","name":"quantity","type":"uint256"}],"name":"order","outputs":[],"stateMutability":"payable","type":"function"}];
//var _abiArray=JSON.parse(_abiStr);

async function doIt() {
    var _order = new web3.eth.Contract(_abiArray, '0x759575cbf8D032b5715a61412fbf7c0e8b729940');
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    var event = _order.events.OrderLog({
            //filter: {_from: accounts[0], _value: [20,50]},
            filter: {_from: accounts[0], _value: 30},
            fromBlock: 'latest', toBlock: 'pending'
        }, function (error, result) {
        if (!error) {
            //console.log("Event fired: " + JSON.stringify(result) + "\n---> " + JSON.stringify(result.returnValues));
            console.log("Event fired: " + JSON.stringify(result.returnValues));
            //process.exit(1);
        }
    });
    var value;
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
    // this will fire an event
    my = await _order.methods.order("0x1234", 3)
        .send({from: accounts[0], gas: 100000, value:30})
        //.then(function(my) {console.log("---> MyFunction called " + JSON.stringify(my.events.OrderLog.returnValues));});
    console.log("---> MyFunction called " + JSON.stringify(my.events.OrderLog.returnValues));
    // this will fire another event
    my = await _order.methods.order("0x1234", 4).send({from: accounts[0], gas: 100000, value:40});
    console.log("---> MyFunction called " + JSON.stringify(my.events.OrderLog.returnValues));
    // this will NOT fire another event
    my = await _order.methods.order("0x1234", 10).send({from: accounts[0], gas: 100000, value:100});
    console.log("---> MyFunction called " + JSON.stringify(my.events.OrderLog.returnValues));
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
}

doIt()


Overwriting src/OrderEventUse.js


첫째 인자는 키를 가지고 있다.


In [ ]:
!node src/OrderEventUse.js

Account: 0x28E9D5b10083b74406b50cbb7B48a299BA6C8Be1
Balance before: 99887306051999993777
Event fired: {"0":"0x28E9D5b10083b74406b50cbb7B48a299BA6C8Be1","1":"0x1234","2":"30","_from":"0x28E9D5b10083b74406b50cbb7B48a299BA6C8Be1","_itemId":"0x1234","_value":"30"}
---> MyFunction called undefined
---> MyFunction called undefined
---> MyFunction called undefined
Balance after: 99885702791999993607
Balance diff: 1603259999993856


## 연습문제 Multiply7

앞서 Multiply7을 구현해 보았다.
이 컨트랙에 이벤트를 추가해서 구현해보자.
아래 주어진 미완성 코드를 구현해서 노드에서 이벤트를 띄워보자.

* 함수는 param4인자를 받아서 곱하기 7 연산을 한다. 8과 16의 연산을 실행하자. 그 결과는 56과 112가 출력될 것이다.
* 이벤트로 넘겨주는 매개변수 param1은 누가 이 함수를 호출했는지 (address), param2는 언제 호출되었는지 (timestamp), 마지막으로 param3는 연산결과를 출력하도록 한다.

```python
contract Multiply7Event {
    event Print(param1, param2, param3);
    function multiply(param4);
}
```


### 단계 1: 컨트랙 개발

```multiply()``` 함수는 ```pure```로 선언되어야 마땅하지만, ```event```를 발생하므로 ```pure``` 또는 ```view```가 될 수가 없다.
따라서 값을 반환하는 것이 의미가 없고, 다음과 같이 
```function multiply(uint input) public``` 선언하는 것이 바람직하다.

In [1]:
%%writefile src/Multiply7Event.sol
pragma solidity ^0.6.0;
contract Multiply7Event {
    event Print(address _addr, uint256 timestamp, uint res);
    //function multiply(uint input) public returns(uint) {
    function multiply(uint input) public {
        uint res = input * 7;
        emit Print(msg.sender, now, res);
        //return res;
    }
}

Overwriting src/Multiply7Event.sol


### 단계 2: 컴파일

In [3]:
!solc --gas --abi --bin src/Multiply7Event.sol


======= src/Multiply7Event.sol:Multiply7Event =======
Gas estimation:
construction:
   111 + 59000 = 59111
external:
   multiply(uint256):	1928
Binary:
6080604052348015600f57600080fd5b506101278061001f6000396000f3fe6080604052348015600f57600080fd5b506004361060285760003560e01c8063c6888fa114602d575b600080fd5b605660048036036020811015604157600080fd5b8101908080359060200190929190505050606c565b6040518082815260200191505060405180910390f35b6000806007830290507f91da4985ab616136202f4e81fd2d8cac1eb12591132d609cece407f7c6fb9205334283604051808473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168152602001838152602001828152602001935050505060405180910390a18091505091905056fea26469706673582212201f2b58641bcadee708439f7e52d3b7d33168278dbdc6285f760ef41356a006df64736f6c63430006010033
Contract JSON ABI
[{"anonymous":false,"inputs":[{"indexed":false,"internalType":"address","name":"_addr","type":"address"},{"indexed":false,"internalType":"uint256","name":"timestamp","type":"ui

### 단계 3: 컨트랙 배포
* 위 abi, bin을 복사해서 붙여 넣는다.

In [92]:
%%writefile src/Multiply7EventDeploy__.js
var Web3=require('web3');
var web3;
if (typeof web3 !== 'undefined') {
    web3 = new Web3(web3.currentProvider);
} else {
    web3 = new Web3(new Web3.providers.HttpProvider("http://117.16.44.45:8445"));
}
var _abiArray=[{"constant":false,"inputs":[{"name":"input","type":"uint256"}],"name":"multiply","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"nonpayable","type":"function"},{"anonymous":false,"inputs":[{"indexed":false,"name":"","type":"uint256"}],"name":"Print","type":"event"}];
var _bin="608060405234801561001057600080fd5b5060fe8061001f6000396000f3fe6080604052348015600f57600080fd5b50600436106045576000357c010000000000000000000000000000000000000000000000000000000090048063c6888fa114604a575b600080fd5b607360048036036020811015605e57600080fd5b81019080803590602001909291905050506089565b6040518082815260200191505060405180910390f35b6000806007830290507f24abdb5865df5079dcc5ac590ff6f01d5c16edbc5fab4e195d9febd1114503da816040518082815260200191505060405180910390a18091505091905056fea165627a7a7230582046cff98a35c11661d6274a85a3b3cf0bb0a0cb33787b6fd675af2741a7009a130029";
var _contract = web3.eth.contract(_abiArray);
//unlock the account with a password provided
//web3.personal.unlockAccount(web3.eth.accounts[0],'password');
var _instance=_contract.new({data:"0x"+_bin,from:web3.eth.accounts[0],gas:1000000}, function(err, contract) {
    if (!err) {
        console.log("contractAddress: ", contract.address);
        console.log("transactionHash: ", contract.transactionHash);
    }
});

Overwriting src/Multiply7EventDeploy.js


In [4]:
%%writefile src/Multiply7EventDeploy.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://117.16.44.45:8345"));
var _abiArray=[{"anonymous":false,"inputs":[{"indexed":false,"internalType":"address","name":"_addr","type":"address"},{"indexed":false,"internalType":"uint256","name":"timestamp","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"res","type":"uint256"}],"name":"Print","type":"event"},{"inputs":[{"internalType":"uint256","name":"input","type":"uint256"}],"name":"multiply","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"nonpayable","type":"function"}];
var _bin="0x" + "6080604052348015600f57600080fd5b506101278061001f6000396000f3fe6080604052348015600f57600080fd5b506004361060285760003560e01c8063c6888fa114602d575b600080fd5b605660048036036020811015604157600080fd5b8101908080359060200190929190505050606c565b6040518082815260200191505060405180910390f35b6000806007830290507f91da4985ab616136202f4e81fd2d8cac1eb12591132d609cece407f7c6fb9205334283604051808473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168152602001838152602001828152602001935050505060405180910390a18091505091905056fea26469706673582212201f2b58641bcadee708439f7e52d3b7d33168278dbdc6285f760ef41356a006df64736f6c63430006010033";
//unlock the account with a password provided
//web3.personal.unlockAccount(web3.eth.accounts[0],'password');
async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin})
        .send({from: accounts[0], gas: 1000000, gasPrice: '1000000000'}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        })
        //.then(function(newContractInstance) {
        //    console.log(newContractInstance.options.address)
        //});
    console.log("---> The contract deployed to: " + deployed.options.address)
}

deploy()


Overwriting src/Multiply7EventDeploy.js


In [5]:
!node src/Multiply7EventDeploy.js

Deploying the contract from 0xe1cD15D96c6109d32B542573f2943878cc0f5b95
hash: 0x10fd61d33819c8c90fabb95a868a1eecdfe87357b80a297750f3235b75fd10cc
---> The contract deployed to: 0x85c1cA1755f706c5228107235f7e5C510e77d392


### 단계 4: 사용

```m7.Print(function (error, result)``` 명령문으로 이벤트를 리스닝하고, 이벤트가 발생하는 경우, 그 결과 ```result```를 출력하고 있다.
이 이벤트는 ```multiply()``` 함수 내에서 발생되고 있다.
그런 까닭에 **```multiply()``` 함수는 ```pure```로 선언할 수 있지만 ```event```를 발생하기 때문에 그렇지 못했다**.
따라서 ```web3```에서 ```transaction```으로 호출해야 하지만. 아래 예제는 **```call()```로 호출**하였다.
```call()``` 함수는 트랜잭션을 발생시키지 않으므로 결과는 알 수 있지만, **이벤트는 발생하지 않는다**.

In [1]:
%%writefile src/Multiply7EventUse__.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://117.16.44.45:8445"));
var _abiArray=[{"constant":false,"inputs":[{"name":"input","type":"uint256"}],"name":"multiply","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"nonpayable","type":"function"},{"anonymous":false,"inputs":[{"indexed":false,"name":"","type":"uint256"}],"name":"Print","type":"event"}];
var _contract = web3.eth.contract(_abiArray);
var m7 = _contract.at("0xee7b78482383b57db71c9c311be3a7ec682042a7");
var event = m7.Print(function (error, result) {
    if (!error) {
        console.log("event Print ===> " + result.args.res);
        console.log("result: ", result);
        process.exit(1);
    }
});
console.log(m7.multiply.call(8).toNumber());

Overwriting src/Multiply7EventUse.js


```multiply()```함수는 state variable을 갱신하지 않으므로, ```call()```로 호출해도 연산 계산을 할 수 있고, 이벤트로 값이 전달된다.


In [6]:
%%writefile src/Multiply7EventUse.js
var Web3=require('web3');
var web3;
if (typeof web3 !== 'undefined') {
    web3 = new Web3(web3.currentProvider);
} else {
    var web3 = new Web3(new Web3.providers.WebsocketProvider("http://117.16.44.45:8345"));
}
var _abiArray=[{"anonymous":false,"inputs":[{"indexed":false,"internalType":"address","name":"_addr","type":"address"},{"indexed":false,"internalType":"uint256","name":"timestamp","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"res","type":"uint256"}],"name":"Print","type":"event"},{"inputs":[{"internalType":"uint256","name":"input","type":"uint256"}],"name":"multiply","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"nonpayable","type":"function"}];
var m7 = new web3.eth.Contract(_abiArray, '0x85c1cA1755f706c5228107235f7e5C510e77d392');
var event = m7.events.Print({fromBlock: 0}, function (error, result) {
    if (!error) {
        //console.log("Event fired: " + JSON.stringify(result) + "\n---> " + JSON.stringify(result.returnValues));
        console.log("Event fired: " + JSON.stringify(result.returnValues));
        //process.exit(1);
    }
});

async function doIt() {
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
    //const value = m7.methods.multiply(8).call();
    const value = m7.methods.multiply(8)
        .send({from: accounts[0]})
        //.then(function(value) {console.log("---> MyFunction called " + JSON.stringify(value) +
        //                               '\n---> '+ JSON.stringify(value.events.Print.returnValues));});
    console.log("---> MyFunction called " + JSON.stringify(value));
        //+ '\n---> '+ JSON.stringify(value.events.Print.returnValues));
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
}

doIt()


Overwriting src/Multiply7EventUse.js


```sendTransaction()``` 함수로 트랜잭션을 발생시켜야 블록체인에 기록이되고, 그 로그에서 이벤트가 발생이 된다.

In [ ]:
!node src/Multiply7EventUse.js

Account: 0xe1cD15D96c6109d32B542573f2943878cc0f5b95
Balance before: 99999611641999999940
---> MyFunction called {"_events":{}}
Balance after: 99999611641999999940
Balance diff: 0
Event fired: {"0":"0xe1cD15D96c6109d32B542573f2943878cc0f5b95","1":"1591257244","2":"56","_addr":"0xe1cD15D96c6109d32B542573f2943878cc0f5b95","timestamp":"1591257244","res":"56"}


In [3]:
!node src/Multiply7EventUse.js

0xb6d5172a1c18f77b3976fa30f24496c805502edb94ed9f564a61da7510b6316b
event Print ===> undefined
result:  { address: '0xee7b78482383b57db71c9c311be3a7ec682042a7',
  blockNumber: 330,
  transactionHash: '0xb6d5172a1c18f77b3976fa30f24496c805502edb94ed9f564a61da7510b6316b',
  transactionIndex: 0,
  blockHash: '0xdd521f6c48f0f95eea5f4b132fbb0de31f78e0e9049c320020ba0c26a0bf2b62',
  logIndex: 0,
  removed: false,
  event: 'Print',
  args: { '': { [String: '56'] s: 1, e: 1, c: [Object] } } }


## 10.5 fallback
<a id='S.3.4'></a>

### fallback
fallback() 함수는 보통 함수와는 다른 기능을 가진다.
사용자가 호출하지 않는다. 따라서 **이름이 없고, 인자가 없고, 반환 값도 없는 함수**이다.
또한 컨트랙에는 **한 개의 fallback 함수**만 존재할 수 있다.
외부에서만 호출할 수 있도록 **external**으로 선언한다.

fallback함수가 없으면 **예외**가 발생하게 된다. 
fallback이 호출되는 경우는:
* 사용자가 호출한 함수가 **존재하지 않을 경우**에 호출된다.
아래에서 ```callA()``` 함수를 호출해야 하는데 잘 못하여 ```callB()```를 호출했다고 하자.
그러면 존재하지 않는 ```callB```()를 실행하지 못하고 ```fallback()```을 실행한다.
* 또한 ether를 송금했는데, 수신측에서 이를 받는 함수가 존재하지 않으면, 송금이 거절되지만 ```fallback```이 호출된다. 이 경우는 **```fallback() payable```**로 선언해야 한다.


### receive
Solidity 버전 0.6.x부터는 ```receive()```함수가 제공되고 있다.
이 함수는 송금을 하지만 calldata가 없는 경우 호출된다.
따라서 ```receive() external payable```로 사용해야 한다.
```receive()```함수가 없으면 ```fallback()```함수가 호출되게 된다.

* receive() external payable은 calldata가 없고, value 필드에 값이 있는 경우
* fallback() external payable은 해당하는 함수가 없는 경우. 송금이 있는 경우에는 payable은 선택적으로, 즉 송금 금액이 있는 경우에 한정적으로 payable을 적어준다.

```python
contract ReceiveTest {
    event PrintLog(address user, uint amount);     
    receive() external payable {
        emit PrintLog(msg.sender, msg.value);
    }
}
```

## 실습: fallback

존재하지 않는 함수를 호출해서 강제적으로 fallback 함수를 실행해 본다.

### 단계 1: 컨트랙 개발

In [1]:
%%writefile src/FallbackTest.sol
pragma solidity ^0.6.0;
contract FallbackTest {
    event PrintLog(string);
    function callA () pure public returns(string memory){
        return "doing callA";
    }
    fallback () external {
        emit PrintLog("fallback called");
    }
}

Overwriting src/FallbackTest.sol


### 단계 2: 컴파일

In [3]:
!solc src/FallbackTest.sol --combined-json abi,bin > src/FallbackTest.json

In [4]:
!cat src/FallbackTest.json

{"contracts":{"src/FallbackTest.sol:FallbackTest":{"abi":"[{\"anonymous\":false,\"inputs\":[{\"indexed\":false,\"internalType\":\"string\",\"name\":\"\",\"type\":\"string\"}],\"name\":\"PrintLog\",\"type\":\"event\"},{\"stateMutability\":\"nonpayable\",\"type\":\"fallback\"},{\"inputs\":[],\"name\":\"callA\",\"outputs\":[{\"internalType\":\"string\",\"name\":\"\",\"type\":\"string\"}],\"stateMutability\":\"pure\",\"type\":\"function\"}]","bin":"608060405234801561001057600080fd5b50610190806100206000396000f3fe608060405234801561001057600080fd5b506004361061002f5760003560e01c8063e7f09e051461009a57610030565b5b7f968f0302429ff0e5bd56a45ce3ba1f4fa79f4b822857e438616435f00c3b59fd60405180806020018281038252600f8152602001807f66616c6c6261636b2063616c6c6564000000000000000000000000000000000081525060200191505060405180910390a1005b6100a261011d565b6040518080602001828103825283818151815260200191508051906020019080838360005b838110156100e25780820151818401526020810190506100c7565b50505050905090810190601f168015610

### 단계 3: 컨트랙 배포

In [4]:
%%writefile src/FallbackTestDeploy__.js
var Web3=require('web3');
var web3;
if (typeof web3 !== 'undefined') {
    web3 = new Web3(web3.currentProvider);
} else {
    web3 = new Web3(new Web3.providers.HttpProvider("http://117.16.44.45:8445"));
}
var _abiArray=[{"constant":true,"inputs":[],"name":"callA","outputs":[{"name":"","type":"string"}],"payable":false,"stateMutability":"pure","type":"function"},{"payable":false,"stateMutability":"nonpayable","type":"fallback"},{"anonymous":false,"inputs":[{"indexed":false,"name":"","type":"string"}],"name":"PrintLog","type":"event"}];
var _bin="608060405234801561001057600080fd5b5061019e806100206000396000f3fe608060405234801561001057600080fd5b5060043610610048576000357c010000000000000000000000000000000000000000000000000000000090048063e7f09e05146100b2575b7f968f0302429ff0e5bd56a45ce3ba1f4fa79f4b822857e438616435f00c3b59fd60405180806020018281038252600f8152602001807f66616c6c6261636b2063616c6c6564000000000000000000000000000000000081525060200191505060405180910390a1005b6100ba610135565b6040518080602001828103825283818151815260200191508051906020019080838360005b838110156100fa5780820151818401526020810190506100df565b50505050905090810190601f1680156101275780820380516001836020036101000a031916815260200191505b509250505060405180910390f35b60606040805190810160405280600b81526020017f646f696e672063616c6c4100000000000000000000000000000000000000000081525090509056fea165627a7a72305820eebb5fe6a9199558a5ae72300540824638cef5eb55bc15bba28146b1b2477f830029";
var _contract = web3.eth.contract(_abiArray);
//unlock the account with a password provided
//web3.personal.unlockAccount(web3.eth.accounts[0],'password');
var _instance=_contract.new({data:"0x"+_bin,from:web3.eth.accounts[0]}, function(err, contract) {
    if (!err) {
        console.log("contractAddress: ", contract.address);
        console.log("transactionHash: ", contract.transactionHash);
    }
});

Writing src/FallbackTestDeploy.js


In [9]:
%%writefile src/FallbackTestDeployFromFile.js
var Web3=require('web3');
var web3=new Web3(new Web3.providers.HttpProvider("http://117.16.44.45:8345"));
var fs=require('fs');
var _str = fs.readFileSync("src/FallbackTest.json");
var _json=JSON.parse(_str)
var _abiArray=JSON.parse(_json.contracts["src/FallbackTest.sol:FallbackTest"].abi);
//console.log(_abiArray);
//var _bin=_json.contracts.sHello2.bin;
var _bin="0x"+_json.contracts["src/FallbackTest.sol:FallbackTest"].bin;
//console.log(_bin);
//unlock the account with a password provided
//web3.personal.unlockAccount(web3.eth.accounts[0],'password');
async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin})
        .send({from: accounts[0], gas: 1000000}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        })
        //.then(function(newContractInstance){
        //    console.log(newContractInstance.options.address)
        //});
    console.log("---> The contract deployed to: " + deployed.options.address)
}
deploy()

Writing src/FallbackTestDeployFromFile.js


In [4]:
!node src/FallbackTestDeployFromFile.js

Deploying the contract from 0xf828946Ec3cD7aDc51a370597D000A5F237bf746
hash: 0xdf389c80fcf3f5407505aaae80c9fe81bf378be9e356bf75e62acbb3bdec06a2
---> The contract deployed to: 0x953C7F383b29C57B605a16ce3360dd7121F7DEB1


### 단계 4: 사용

REMIX에서는 fallback 함수를 호출할 수 있도록 노출하지 않는다.
REMIX 버전 0.9.3부터 지원된 **Low level interactions**라는 기능을 통해 fallback 함수를 호출할 수 있다.
**최소 1바이트 이상을 입력해야 하는데, 16진수로 calldata를 입력**하고 ```Transact``` 버튼을 누르면,
우측 아래 REMIX 콘솔 창에 함수가 호출되었다는 로그가 뜬다.
로그를 열어 보면 이벤트가 발생하였다는 것을 확인할 수 있다.

**아무 데이터도 입력하지 않고** ```Transact``` 버튼을 누르면, ```receive```가 호출된다?

![alt text](figures/10_fallbackREMIX.png "call fallback via low level interactions on REMIX")

In [3]:
%%writefile src/FallbackTestUse__.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://117.16.44.45:8445"));
var _abiArray=[{"constant":true,"inputs":[],"name":"callA","outputs":[{"name":"","type":"string"}],"payable":false,"stateMutability":"pure","type":"function"},{"payable":false,"stateMutability":"nonpayable","type":"fallback"},{"anonymous":false,"inputs":[{"indexed":false,"name":"","type":"string"}],"name":"PrintLog","type":"event"}];
var _contract = web3.eth.contract(_abiArray);
var fallback = _contract.at("0x3ab1841c257571342e04b0fc898d2e63050585b2");
var filter = fallback.PrintLog(function (error, result) {
  if (!error)
    console.log(result);
});
web3.eth.sendTransaction({from:web3.eth.accounts[0], to:"0x3ab1841c257571342e04b0fc898d2e63050585b2"});

Overwriting src/FallbackTestUse.js


In [5]:
%%writefile src/FallbackTestUseFromFile.js
var Web3=require('web3');
var web3=new Web3(new Web3.providers.HttpProvider("http://117.16.44.45:8345"));
var fs=require('fs');
var _str = fs.readFileSync("src/FallbackTest.json");
var _json=JSON.parse(_str)
var _abiArray=JSON.parse(_json.contracts["src/FallbackTest.sol:FallbackTest"].abi);
var _instance = new web3.eth.Contract(_abiArray, "0x953C7F383b29C57B605a16ce3360dd7121F7DEB1");
var event = _instance.events.PrintLog(function (error, result) {
    if (!error) {
        console.log("Event fired: " + JSON.stringify(result) + "\n---> " + JSON.stringify(result.returnValues));
    }
});

async function doIt() {
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
    _instance.methods.callA().call().then(console.log);  //null
    //call without calling any method
    //await _instance.methods.callB().send({from:accounts[0], to: 0x08c6FB3fFDfdf5d46db22A7e9e8Ac8DdC930bbC9});
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
}

doIt()

Overwriting src/FallbackTestUseFromFile.js


#### 존재하지 않는 function selector

**존재하지 않는 함수를 호출**하면 fallback함수가 호출된다.
클라이언트에서 web3를 사용하면, 존재하지 않는 함수를 호출할 수 있는 방법은 없다.
가능한 방법은 ```web3.eth.sendTransaction()```의 ```data```, 즉 ```msg.data```를 수정하면 된다. 앞서 설명한 **function selector** (호출할 함수시그니처의 sha3 해시값으로 만들고, 앞 4바이트)가 존재하지 않도록 수정해야 한다.

```python
<address>.call(bytes4(bytes32(sha3("baz(uint32,bool)")))
```

아쉬운대로 다음과 같이 **함수명을 생략하여 강제적으로 존재하지 않는 함수**를 호출한다.
즉 컨트랙 주소로 그냥 트랜잭션을 전송하여 없는 함수가 호출된 것처럼 한다.

```python
web3.eth.sendTransaction({from:web3.eth.accounts[0], to:<<contract address>>});
```

In [6]:
!node src/FallbackTestUseFromFile.js

Account: 0xf828946Ec3cD7aDc51a370597D000A5F237bf746
Balance before: 99996822860000000000
doing callA
Balance after: 99996822860000000000
Balance diff: 0


In [ ]:
!node src/FallbackTestUse.js

{ address: '0x3ab1841c257571342e04b0fc898d2e63050585b2',
  blockNumber: 45484,
  transactionHash: '0x5725425ea888cfcad16298c2e4a1d3fd71cbb2880230b82762ac5be12fee7def',
  transactionIndex: 0,
  blockHash: '0xe4e9f371bb90f67279e9284c17451cba53b5eb850ed416249fe32b64f801e20c',
  logIndex: 0,
  removed: false,
  event: 'PrintLog',
  args: { '': 'fallback called' } }


## 실습: payable fallback

앞서 존재하지 않는 함수를 호출하는 경우, ```fallback```함수가 호출되었다.
또한 금액없이 송금을 하여도 ```fallback```함수가 호출된다.

송금을 하면서 오류가 발생하여 ```fallback```이 호출된다 하더라도 송금액은 받을 수 없다.
단, ```payable fabllback```으로 선언하면 송금액을 받을 수 있다.

```python
function () payable {}
```


### 단계 1: 컨트랙 개발

```fallback()``` 함수가 호출되려면, web3.js 측에서가 아니라 다른 컨트랙에서 실험하는 편이 좋다.
그렇다면 부득이 컨트랙을 하나 더 만들어야 하고, 두 컨트랙이 연관을 가져야 한다.
아래 코드를 보면:
* 두 컨트랙이 하나의 파일에 존재하면 ```new``` 명령어로 객체를 만들어 사용하거나,
* 이미 배포된 컨트랙을 사용하려고 하면, 그 주소를 할당하여 사용하고 있다.

In [55]:
%%writefile src/MathMultiply7.sol
pragma solidity ^0.6.0;

contract Multiply7 {
   //event Print(uint);
   receive() external payable {}
   fallback() external payable {}
   function multiply(uint input) pure public returns (uint) {
      //emit Print(input * 7);
      return input * 7;
   }
   function getAddress() view public returns(address) {
       return address(this);
   }
}

contract Math {
    Multiply7 m7=new Multiply7();
    function deposit(uint amount) payable public {
        require(msg.value==amount);
    }
    function setM7(address payable _addr) public { m7 = Multiply7(_addr); }
    function multiply() view public returns(uint) {
        uint res=m7.multiply(8);
        //this.send11(); It does not send value.
        return res;
    }
    function send11M7() public payable {
        //m7.multiply.value(11)(9);
        //m7.multiply(9);
        address(m7).transfer(11);
    }
    function getBalanceOfThis() public view returns(uint) {
        return address(this).balance;
    }
    function getBalanceOfM7() public view returns(uint) {
        return address(m7).balance;
    }
    function getAddressOfM7() view public returns(address) {
        return address(m7);
   }    
}

Overwriting src/MathMultiply7.sol


### 단계 2: 컴파일

지금은 버전 0.6으로 컴파일해서 실행하고 있다. 실행에 문제가 있으면 REMIX에서 버전을 0.4.21로 낮추어 적용하여 보자.

In [56]:
!solc src/MathMultiply7.sol --combined-json abi,bin > src/MathMultiply7.json

In [59]:
!cat src/MathMultiply7.json

{"contracts":{"src/MathMultiply7.sol:Math":{"abi":"[{\"inputs\":[{\"internalType\":\"uint256\",\"name\":\"amount\",\"type\":\"uint256\"}],\"name\":\"deposit\",\"outputs\":[],\"stateMutability\":\"payable\",\"type\":\"function\"},{\"inputs\":[],\"name\":\"getAddressOfM7\",\"outputs\":[{\"internalType\":\"address\",\"name\":\"\",\"type\":\"address\"}],\"stateMutability\":\"view\",\"type\":\"function\"},{\"inputs\":[],\"name\":\"getBalanceOfM7\",\"outputs\":[{\"internalType\":\"uint256\",\"name\":\"\",\"type\":\"uint256\"}],\"stateMutability\":\"view\",\"type\":\"function\"},{\"inputs\":[],\"name\":\"getBalanceOfThis\",\"outputs\":[{\"internalType\":\"uint256\",\"name\":\"\",\"type\":\"uint256\"}],\"stateMutability\":\"view\",\"type\":\"function\"},{\"inputs\":[],\"name\":\"multiply\",\"outputs\":[{\"internalType\":\"uint256\",\"name\":\"\",\"type\":\"uint256\"}],\"stateMutability\":\"view\",\"type\":\"function\"},{\"inputs\":[],\"name\":\"send11M7\",\"outputs\":[],\"stateMutability\":\"p

### 단계 3: 컨트랙 배포

In [45]:
%%writefile src/MathMultiply7Deploy__.js
var Web3=require('web3');
var web3;
if (typeof web3 !== 'undefined') {
    web3 = new Web3(web3.currentProvider);
} else {
    web3 = new Web3(new Web3.providers.HttpProvider("http://117.16.44.45:8445"));
}
//taken from REMIX from here
var mathContract = web3.eth.contract([{"constant":true,"inputs":[],"name":"queryBalance","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"send11","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":true,"inputs":[],"name":"queryBalanceM7","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"multiply","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"_addr","type":"address"}],"name":"m7set","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"}]);
var math = mathContract.new(
   {
     from: web3.eth.accounts[0], 
     data: '0x606060405261000c610071565b604051809103906000f080151561002257600080fd5b6000806101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff160217905550341561006c57600080fd5b610081565b604051610105806103e883390190565b610358806100906000396000f300606060405260043610610078576000357c0100000000000000000000000000000000000000000000000000000000900463ffffffff16806336f40c611461007d5780633c58d09e146100a6578063b6b55f25146100cf578063b76d4d06146100e7578063f3593cd014610110578063fde3137f14610139575b600080fd5b341561008857600080fd5b610090610172565b6040518082815260200191505060405180910390f35b34156100b157600080fd5b6100b9610191565b6040518082815260200191505060405180910390f35b6100e560048080359060200190919050506101e5565b005b34156100f257600080fd5b6100fa6101f6565b6040518082815260200191505060405180910390f35b341561011b57600080fd5b610123610236565b6040518082815260200191505060405180910390f35b341561014457600080fd5b610170600480803573ffffffffffffffffffffffffffffffffffffffff169060200190919050506102e9565b005b60003073ffffffffffffffffffffffffffffffffffffffff1631905090565b60008060009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff16600b60405160006040518083038185875af1925050505090565b80341415156101f357600080fd5b50565b60008060009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1631905090565b6000806000809054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1663c6888fa160086040518263ffffffff167c010000000000000000000000000000000000000000000000000000000002815260040180828152602001915050602060405180830381600087803b15156102c957600080fd5b5af115156102d657600080fd5b5050506040518051905090508091505090565b806000806101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff160217905550505600a165627a7a72305820ca83e1f4743e5ee693d006254f4b0cb550b1e449bc6a95c89e2b575f901cd26300296060604052341561000f57600080fd5b60e88061001d6000396000f300606060405260043610603f576000357c0100000000000000000000000000000000000000000000000000000000900463ffffffff168063c6888fa1146041575b005b3415604b57600080fd5b605f60048080359060200190919050506075565b6040518082815260200191505060405180910390f35b60007f24abdb5865df5079dcc5ac590ff6f01d5c16edbc5fab4e195d9febd1114503da600783026040518082815260200191505060405180910390a16007820290509190505600a165627a7a72305820f19d1151fc4778023048c2f305ffb042231305525da155ac10d2fdd0112e607b0029', 
     gas: '4700000'
   }, function (e, contract){
    //console.log(e, contract);
    if (typeof contract.address !== 'undefined') {
         console.log('Contract mined! address: ' + contract.address + ' transactionHash: ' + contract.transactionHash);
    }
 })

Overwriting src/MathMultiply7Deploy.js


In [60]:
%%writefile src/MathMultiply7DeployFromFile.js
var Web3=require('web3');
var web3=new Web3(new Web3.providers.HttpProvider("http://117.16.44.45:8345"));
var fs=require('fs');
var _str = fs.readFileSync("src/MathMultiply7.json");
var _json=JSON.parse(_str)
var _abiArray=JSON.parse(_json.contracts["src/MathMultiply7.sol:Math"].abi);
//console.log(_abiArray);
//var _bin=_json.contracts.sHello2.bin;
var _bin="0x"+_json.contracts["src/MathMultiply7.sol:Math"].bin;
//console.log(_bin);
//unlock the account with a password provided
//web3.personal.unlockAccount(web3.eth.accounts[0],'password');
async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin})
        .send({from: accounts[0], gas: 1000000}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        })
        //.then(function(newContractInstance){
        //    console.log(newContractInstance.options.address)
        //});
    console.log("---> The contract deployed to: " + deployed.options.address)
}
deploy()

Overwriting src/MathMultiply7DeployFromFile.js


In [61]:
!node src/MathMultiply7DeployFromFile.js

Deploying the contract from 0xd2d45Dd6457A410b86632c4Fb9b2E9535DD61e55
hash: 0x4b6495636a0ad52f13acd2fb66e01bb13d497eaffc7934b83b3113472e6678d5
---> The contract deployed to: 0x4Dc626EEB7CDDBfDD631D25407FEcAe6510F1305


### 단계 4: 사용

### 컨트랙 연관관계에서의 주소 설정

```new``` 명령어는 동일한 파일에 연관관계 컨트랙이 존재할 때 주소를 할당하여 객체를 생성한다.
```setM7(address)``` 함수는 ```Multiply7```를 배포하고, 확득한 주소를 할당한다.

### 입금해서 잔고가 있어야 송금 가능: send11M7()

9를 곱하면서 ```11 wei```를 송금하려면 ```m7.multiply.value(11)(9)```로 호출할 수 있다.
송금하면서 ```payable fabllback```를 호출되게 하기 위해 의도적으로 아래와 같이 존재하지 않는 함수를 호출하였다.

```python
call.value()();
```

송금하려면 물론 ```deposit()```을 먼저해야 한다.

1. ```deposit(123)```
value를 반드시 함수인자와 동일하게 입력해야 한다.
REMIX에서 할 경우에도 마찬가지이다. ```Run``` 탭 상단의 ```value```, ```Deployed Contracts``` 함수의 인자 두 필드에 동일한 금액을 넣어준다.
2. ```getBalanceOfThis()``` --> 123 입금하고 난 후의 잔고
3. ```queryBalanceOfM7()``` ---> 0
4. ```send11()```
존재하지 않는 함수를 호출하여 ```m7.call.value(11)()```,
다른 컨트랙 ```Multiply7```의 ```payable fallback``` 함수를 호출하여 ```11 Wei```를 송금한다.
5. ```getBalanceOfThis()``` ---> 112
6. ```getBalanceOfM7()``` ---> 11 ```payable fallback``` **존재하지 않는 함수를 호출하는 오류에도 불구하고 11 wei 송금 성공**

```python
> var math = mathContract.at("0xf803dac95ea40f03736060114dfb9cca9ee7d514");
undefined
> math.deposit.sendTransaction(123,{from:web3.eth.accounts[0], value:123});   value와 인자를 동일하게 123으로 입금
'0x9aa405b5780c999755ae85c3bc852a1e3b09a7d29b6fea20cdfb1af2ac4da19d'          REMIX에서도 마찬가지로 value와 인자를 일치시킴.
> //miner.start(1);admin.sleepBlocks(1);miner.stop();
undefined
> math.queryBalance.call().toNumber();                                        마이닝하고 나면 잔고 123
123
> math.queryBalanceM7.call().toNumber();                                      Multiply7 잔고는 0
0
> math.send11({from:web3.eth.accounts[0]});                                   call.value(11)로 송금
'0x9c497939df2a5d37cc7f9bf29b0f149831b82bfb841c2163badd360cba5f3360'
> //miner.start(1);admin.sleepBlocks(1);miner.stop();
undefined
> math.queryBalance.call().toNumber();                                        마이닝하고 나면 잔고 112=123-11
112
> math.queryBalanceM7.call().toNumber();                                      Multiply7 잔고는 11증가
11
> math.multiply.call().toNumber();                                            함수내 8은 이미 입력되어 있으므로 56 = 8 x 7
56
```

In [4]:
%%writefile src/MathMultiply7Use__.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://117.16.44.45:8445"));
var mathContract = web3.eth.contract([{"constant":true,"inputs":[],"name":"queryBalance","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"send11","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":true,"inputs":[],"name":"queryBalanceM7","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"multiply","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"_addr","type":"address"}],"name":"m7set","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"}]);
var math = mathContract.at("0xf803dac95ea40f03736060114dfb9cca9ee7d514");
console.log("deposit 123..."+math.deposit.sendTransaction(123,{from:web3.eth.accounts[0], value:123}));
console.log("math balance: "+math.queryBalance.call().toNumber());
math.send11({from:web3.eth.accounts[0]});
console.log("math balance: "+math.queryBalance.call().toNumber());
console.log("m7 balance: "+math.queryBalanceM7.call().toNumber());
math.multiply.call().toNumber();

Overwriting src/MathMultiply7Use.js


In [72]:
%%writefile src/MathMultiply7UseFromFile.js
var Web3=require('web3');
var web3=new Web3(new Web3.providers.HttpProvider("http://117.16.44.45:8345"));
var fs=require('fs');
var _str = fs.readFileSync("src/MathMultiply7.json");
var _json=JSON.parse(_str)
var _abiArray=JSON.parse(_json.contracts["src/MathMultiply7.sol:Math"].abi);
var _instance = new web3.eth.Contract(_abiArray, "0x4Dc626EEB7CDDBfDD631D25407FEcAe6510F1305");

async function doIt() {
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
    _instance.methods.multiply().call().then(console.log);
    _instance.methods.deposit(123).send({from:accounts[0], value:123});
    _instance.methods.getBalanceOfM7().call().then(console.log);
    await _instance.methods.send11M7().send({from:accounts[0]});
    _instance.methods.getBalanceOfM7().call().then(console.log);
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
}

doIt()

Overwriting src/MathMultiply7UseFromFile.js


In [73]:
!node src/MathMultiply7UseFromFile.js

Account: 0xd2d45Dd6457A410b86632c4Fb9b2E9535DD61e55
Balance before: 99982252519999999754
56
0
11
Balance after: 99981233199999999631
Balance diff: 1019320000004096


In [74]:
!node src/MathMultiply7UseFromFile.js

Account: 0xd2d45Dd6457A410b86632c4Fb9b2E9535DD61e55
Balance before: 99981233199999999631
56
11
22
Balance after: 99980213879999999508
Balance diff: 1019319999987712


## 연습문제

블록체인에서 주문을 하고, 그 내역을 로컬 파일에 작성해 보자.
블록체인으로 주문을 전송하면, 이벤트가 발생하고 주문내역을 로컬파일에 쓰게 된다.
* ```EventTest.sol```에서는 이벤트가 발생하면서 그 로그를 파일에 작성한다 (```OrderEvent.txt```)
* ```OrderEvent.sol```을 수정하여:
	* ```order(상품항목, 주문 개수, 배송지)``` 함수 작성
	* ```getBalance()``` 함수 작성. 해당 프로그램 생성한 ```owner```만 읽을 수 있게 ```modfier isOnwer``` 작성.

```geth``` 또는 ```ganache``` 어느 하나를 선택하여 프로그램을 작성하세요.
아래 로그가 파일에 쓰여지도록 하고, 노트북에서 그 파일을 출력하세요 (```!cat src/OrderEvent.txt```, 윈도우에서는 ```cat``` 대신 ```type```)
```python
전송자주소, 1111, 3, 20 2-gil Hongji-dong Jongro-gu Seoul
전송자주소, 1111, 5, 20 2-gil Hongji-dong Jongro-gu Seoul
전송자주소, 1111, 20, 20 2-gil Hongji-dong Jongro-gu Seoul
```